In [3]:
import os, sys, matplotlib, pylab
import numpy  as np
from astropy.io import fits
from matplotlib import pyplot as plt
import re
from astropy.table import Table, Column
from astropy.io import ascii


def norm(a):
    p = (a-np.median(a))/np.median(a)
    return p


In [5]:
#Read folder name starting with ktwo
os.chdir("/home/vatsal/UThesis/K2/data/kepler") # Change Directory
nowpath = os.getcwd()
filenamelist = filter(lambda x: x.startswith('ktwo'), os.listdir(nowpath))
writelist = [ i +'\n' for i in filenamelist]
with open('list.txt','w') as f:
    map( f.write , writelist)
f.close()                                       #making the folder list

Q10up = 'list.txt'
Q10_file = file(Q10up).readlines()

ktwoID = []
ktwofilelist = []

ktwo_detID = []
ktwo_detfilelist = []

for i in Q10_file:
    path = i.replace('\n','')
    os.chdir("/home/vatsal/UThesis/K2/data/kepler/%s"%path)
    
    foldername = i.strip().split(' ')
    folderlistfile = '%s_list.txt' %foldername[0]
    
    
    #1 Read names of all files ending with .fits
    nowpath = os.getcwd()
    filenamelist = filter(lambda x: x.endswith('.fits'), os.listdir(nowpath))
    
    #1.5 Write Raw data file into txt list (.fits)
    writelist = [ i +'\n' for i in filenamelist]
    with open('%s_Fits_list.txt' %foldername[0],'w') as f:
        map(f.write , writelist)
        
    #2 Read needed file data
    Kepler = '%s_Fits_list.txt' %foldername[0]
    LC_file= file(Kepler).readlines()
    
    
    
    for j in LC_file:
        if(j.startswith('ktwo')):
            a = re.findall('\d+', j)
            ktwoID.append(a[0])
            ktwofilelist.append(j)
        
        elif(j.startswith('EPIC')):
            a = re.findall('\d+', j)
            ktwo_detID.append(a[0])
            ktwo_detfilelist.append(j)

com_ID =  [i for i in set(ktwoID) & set(ktwo_detID)]      #list of IDs common to both original and detrended list of files-
                                                          #essentially the ones that have to be analysed




#Make a list of tuples containing names of common files from both folder

ktwofilelist = zip([np.float(k) for k in ktwoID],ktwofilelist)
ktwo_detfilelist = zip([np.float(l) for l in ktwo_detID],ktwo_detfilelist)


ktwofilelist.sort(key=lambda x: x[0])
ktwo_detfilelist.sort(key=lambda x: x[0])

print len(ktwofilelist)
print len(ktwo_detfilelist)

comlist = zip([k[1] for k in ktwofilelist],[k[1] for k in ktwo_detfilelist])

print comlist


os.chdir("/home/vatsal/UThesis/K2/data/kepler")
nowpath = os.getcwd()

#Defining the pattern to read the kernel hyperparameter values
numeric_const_pattern = r"""
[-+]? # optional sign
(?:
(?: \d* \. \d+ ) # .1 .12 .123 etc 9.1 etc 98.1 etc
|
(?: \d+ \.? ) # 1. 12. 123. etc 1 12 123 etc
 )
# followed by optional exponent part if desired
(?: [Ee] [+-]? \d+ ) ?
"""
rx = re.compile(numeric_const_pattern, re.VERBOSE)
rot = [] #To store the rotation periods

for k,l in comlist:
    
    keplerid = re.findall('\d+',k)[0]
    k = k.replace('\n','')
    data1 = fits.open(nowpath+'/ktwo/'+k)
    dat=data1[1].data
    pdcsap_flux = dat['PDCSAP_FLUX']
    time_raw = dat['TIME']
    
    mnan1 = -np.isnan(pdcsap_flux)
    
    
    l = l.replace('\n','')
    data2 = fits.open(nowpath+'/ktwo-detrended/'+l)
    dat=data2[1].data
    time_det = dat['time']
    flux_det = dat['flux']
    trend_t = dat['trtime']
    trend_p = dat['trposi']
    x_cor = dat['x']
    y_cor = dat['y']
    
    mnan2 = -np.isnan(flux_det)
    
#     output_file = l.replace('\n','') + '.rdb'
#     fdata = Table([time_det,flux_det, trend_t, trend_p], names=['jdb', 'flux', 'trendt','trendp'])
#     ascii.write(fdata,'/home/vatsal/UThesis/K2/data/kepler/ktwo-detrended/'+output_file , format='rdb')
#     #rot.append(rx.findall(data2[1].header['KER_HPS1'])[2])
    
    
#     fig, ax = plt.subplots(4, sharex=True)
    
#     #Plotting the Raw flux from original ktwo files
#     ax[0].plot(time_raw[mnan1],100.*norm(pdcsap_flux[mnan1]),'k.-')
#     ax[0].set_ylabel('Raw Relative flux[%]')
#     ax[0].set_title(keplerid+' '+'Rotation period:'+rx.findall(data2[1].header['KER_HPS1'])[2])
    
#     #Plotting the supposedly detrended flux from the EPIC files
#     ax[1].plot(time_det[mnan2],100.*norm(flux_det[mnan2]),'k.-')
#     ax[1].set_ylabel('f[%]')
    
#     #Plotting the light curves with corrections for systematics
#     ax[2].plot(time_det[mnan2],100.*norm(flux_det[mnan2])-100.*norm(trend_p[mnan2]),'k.-')
#     ax[2].plot(time_det[mnan2],100.*norm(trend_t[mnan2]),'r')
#     ax[2].set_ylabel('f-m(X)[%]')
    
#     #Plotting the light curves with corrections for both systematics and intrinsic stellar variations
#     ax[3].plot(time_det[mnan2],100.*norm(flux_det[mnan2])-100.*norm(trend_p[mnan2])-100.*norm(trend_t[mnan2]),'k.-')
#     ax[3].set_ylabel('f-m(X)-m(t)[%]')
#     ax[3].set_xlabel('Time')
    
#     plt.show()

    

285
255
[('ktwo205911733-c03_llc.fits\n', 'EPIC_205911733_mast.fits\n'), ('ktwo205945160-c03_llc.fits\n', 'EPIC_205945160_mast.fits\n'), ('ktwo206030934-c03_llc.fits\n', 'EPIC_206030934_mast.fits\n'), ('ktwo206050032-c03_llc.fits\n', 'EPIC_206050032_mast.fits\n'), ('ktwo206053352-c03_llc.fits\n', 'EPIC_206053352_mast.fits\n'), ('ktwo206076184-c03_llc.fits\n', 'EPIC_206076184_mast.fits\n'), ('ktwo206101438-c03_llc.fits\n', 'EPIC_206101438_mast.fits\n'), ('ktwo206112518-c03_llc.fits\n', 'EPIC_206112518_mast.fits\n'), ('ktwo206131233-c03_llc.fits\n', 'EPIC_206131233_mast.fits\n'), ('ktwo206135809-c03_llc.fits\n', 'EPIC_206135809_mast.fits\n'), ('ktwo206145704-c03_llc.fits\n', 'EPIC_206145704_mast.fits\n'), ('ktwo206172696-c03_llc.fits\n', 'EPIC_206172696_mast.fits\n'), ('ktwo206181579-c03_llc.fits\n', 'EPIC_206181579_mast.fits\n'), ('ktwo206213179-c03_llc.fits\n', 'EPIC_206213179_mast.fits\n'), ('ktwo206305683-c03_llc.fits\n', 'EPIC_206305683_mast.fits\n'), ('ktwo206343236-c03_llc.fits\n'

In [6]:
##Finding the important things out of the mess (2300<Teff<3000 for now)

n = [np.int(i) for i in com_ID]
n.sort()
m = [str(i) for i in n]
#print m

dt = np.loadtxt('/home/vatsal/UThesis/K2/catalog/K2-greater-than-2300.rdb') 
k2 = dt[:,0]
k2 = [str(int(i)) for i in k2]
Teff = dt[:,1]
logg = dt[:,2]
R = dt[:,3]
Mass = dt[:,4]


k2 = sorted(k2)

coh = set(k2).intersection(m)
coh = list(coh)
print len(coh)

#Get the indices of elements in k2 that are present in coh

indices = [i for k in np.arange(len(coh)) for i in np.arange(len(k2)) if k2[i]==coh[k] ]
indices = sorted(indices)
k21 = [k2[i] for i in indices]
coh1 = sorted(coh)
plt.plot(k21,coh1,'.')
plt.show()


# print np.arange(len(coh))

alt = zip(coh1,[Teff[i] for i in indices],[logg[i] for i in indices],[R[i] for i in indices],[Mass[i] for i in indices])
print alt      #This list of tuples contains all the IDs and their info, for targets>2300K, which have been detrended
               #and their 
print len(alt)


# with open('/home/vatsal/UThesis/K2/catalog/K2-greater-than-2300-fin.txt','w') as f:
#     map(f.write , [i+'\n' for i in coh])

# fdata2 = Table([m,rot], names=['Kepler_ID','P_rot'])
# ascii.write(fdata2,'/home/vatsal/UThesis/K2/results/K2-period.txt' , format='rdb')

142
[('206050032', 2617.0, 5.2709999999999999, 0.113, 0.089999999999999997), ('206076184', 2719.0, 5.2430000000000003, 0.11799999999999999, 0.094), ('206131233', 2427.0, 5.1120000000000001, 0.108, 0.079000000000000001), ('206135809', 2418.0, 5.2480000000000002, 0.106, 0.083000000000000004), ('206181579', 2991.0, 5.2009999999999996, 0.14299999999999999, 0.121), ('206213179', 2959.0, 5.2060000000000004, 0.14199999999999999, 0.11799999999999999), ('206358491', 2972.0, 5.2050000000000001, 0.14199999999999999, 0.11899999999999999), ('206376197', 2977.0, 5.194, 0.14599999999999999, 0.121), ('206381460', 2357.0, 5.3109999999999999, 0.10299999999999999, 0.081000000000000003), ('206386821', 2316.0, 5.327, 0.10100000000000001, 0.079000000000000001), ('206494490', 2849.0, 5.2329999999999997, 0.13, 0.105), ('210311375', 2842.0, 5.226, 0.129, 0.106), ('210456477', 2993.0, 5.2130000000000001, 0.13900000000000001, 0.11899999999999999), ('210467490', 2421.0, 5.2850000000000001, 0.111, 0.14499999999999

In [16]:
#unzipping the values of KIDs and their properties for calculating the energy 
k_id, T_eff, log_g, R_s, M_s   = map(list, zip(*alt))

flist = ['EPIC_'+i+'_mast.fits.rdb' for i in k_id]
print len(flist)

142


Counting flares

In [158]:
# kl = file('/home/vatsal/UThesis/K2/data/kepler/ktwo-detrended/ktwo-detrended_Fits_list.txt').readlines()

# kli = [i.replace('\n','') for i in kl]
# flist = [i+'.rdb' for i in kli]

# print flist
# print len(flist)

['EPIC_211160943_mast.fits.rdb', 'EPIC_206305683_mast.fits.rdb', 'EPIC_217509281_mast.fits.rdb', 'EPIC_219306155_mast.fits.rdb', 'EPIC_219057545_mast.fits.rdb', 'EPIC_211997901_mast.fits.rdb', 'EPIC_211435030_mast.fits.rdb', 'EPIC_211711533_mast.fits.rdb', 'EPIC_211792422_mast.fits.rdb', 'EPIC_211801135_mast.fits.rdb', 'EPIC_211028347_mast.fits.rdb', 'EPIC_206494490_mast.fits.rdb', 'EPIC_211634376_mast.fits.rdb', 'EPIC_212022746_mast.fits.rdb', 'EPIC_206381460_mast.fits.rdb', 'EPIC_212531681_mast.fits.rdb', 'EPIC_212430908_mast.fits.rdb', 'EPIC_212814999_mast.fits.rdb', 'EPIC_212614976_mast.fits.rdb', 'EPIC_214733184_mast.fits.rdb', 'EPIC_211046195_mast.fits.rdb', 'EPIC_211963497_mast.fits.rdb', 'EPIC_210911931_mast.fits.rdb', 'EPIC_214394595_mast.fits.rdb', 'EPIC_212694426_mast.fits.rdb', 'EPIC_211494804_mast.fits.rdb', 'EPIC_211962038_mast.fits.rdb', 'EPIC_213826072_mast.fits.rdb', 'EPIC_210311375_mast.fits.rdb', 'EPIC_212716937_mast.fits.rdb', 'EPIC_219745741_mast.fits.rdb', 'EPIC_2

In [20]:
binsize = 4

countlist = []

for s in flist:
    
    f = open('/home/vatsal/UThesis/K2/data/kepler/ktwo-detrended/'+s, 'r')  

    d = f.read()
    table = ascii.read(d)
    time_det = np.float64(table['jdb'][1:])
    flux_det = np.float64(table['flux'][1:])
    trend_t = np.float64(table['trendt'][1:])
    trend_p = np.float64(table['trendp'][1:])



    f.close()


    #mask the nan point
    # masknan = -np.isnan(data['flux'])
    # time = data['time'][masknan]
    # flux = data['flux'][masknan]/np.mean(data['flux'][masknan])
    # flux_raw = data['flux'][masknan]/np.mean(data['flux'][masknan])


    mnanm = -np.isnan(flux_det)
    time = time_det[mnanm]
    flux_d = 100.*norm(flux_det[mnanm])-100.*norm(trend_p[mnanm])-100.*norm(trend_t[mnanm])
    flux_r = 100.*norm(flux_det[mnanm])-100.*norm(trend_p[mnanm])-100.*norm(trend_t[mnanm])

    #make empty array as same length as masked flux
    fluxC = np.zeros(len(flux_d))


    #separate gap when time step > 3*time step 
    separationtime = []
    separationtime.append(0)
    for i in range(len(time)-1):
        if time[i+1] - time[i] > 3*(time[2] - time[1]):
            separationtime.append(i)
    separationtime.append(len(time))

    #print separationtime

    #do j times
    for j in range(2):
        for k in range(len(separationtime)-1):
            #take 10 point median and replace it
            for i in range(separationtime[k],separationtime[k+1]):
                if i >= (separationtime[k]+binsize) and i <= (separationtime[k+1]-binsize): 
                    fluxC[i] = np.median(flux_d[i-binsize:i+binsize])
                else :
                    fluxC[i] = flux_d[i]


            #Observed subtract Caculate 

            fluxdiff = flux_d-fluxC

            #move out the peak value > 1 sigma
            maskpeak = fluxdiff > 1.0*fluxdiff.std()

            #if the peak larger than standard deviation then replace the number by 10 point median
            for n, i in enumerate(maskpeak):
                if i:
                    if n >= (separationtime[k]+binsize) and n <= (separationtime[k+1]-binsize):
                        flux_d[n] = np.median(flux_d[n-binsize:n+binsize])
                    else:
                        flux_d[n] = flux_d[n]


    #Plotting the O-C flux

    OminusC = flux_r - fluxC

    #plt.plot(time,flux_raw,'o-')
    #plt.plot(time,fluxC,'o-')
    #     plt.plot(time,OminusC,'.-')
    #     plt.show()

    flare = np.where(OminusC > 3.0*OminusC.std())

    #############

    OminusC = flux_r-fluxC

    flares = np.zeros(len(OminusC))

    flareselection = OminusC > 3.0*OminusC.std()
    #print 3.0*OminusC.std()
    #print 3.0*fluxC.std()

    for n, i in enumerate(flareselection):
        if i:
            if OminusC[n-2] > 0.05*OminusC[n]:
                flares[n-2] = OminusC[n-2]

            if OminusC[n-1] > 0.05*OminusC[n]:
                flares[n-1] = OminusC[n-1]

            flares[n] = OminusC[n]

            if OminusC[n] > OminusC[n+1] or OminusC[n+1] > 0.05*OminusC[n]:
                flares[n+1] = OminusC[n+1]

            if OminusC[n+1] > OminusC[n+2] or OminusC[n+2] > 0.05*OminusC[n]:
                flares[n+2] = OminusC[n+2]

            if OminusC[n+2] > OminusC[n+3] or OminusC[n+3] > 0.05*OminusC[n]:
                flares[n+3] = OminusC[n+3]

            if n+4 < len(flareselection):
                if OminusC[n+3] > OminusC[n+4] or OminusC[n+4] > 0.05*OminusC[n]:
                    flares[n+4] = OminusC[n+4]

            if n+5 < len(flareselection):
                if OminusC[n+4] > OminusC[n+5] or OminusC[n+5] > 0.05*OminusC[n]:
                    flares[n+5] = OminusC[n+5]

    #change all minus value to 0

    for i in range(len(flares)):
        if flares[i] < 0:
            flares[i] = 0

    #remove fake flares

    for i in range(len(flares)-1):        
        if  flares[i] < 3.0*fluxC.std() and \
            flares[i-1] == 0 and \
            flares[i+1] == 0 :
                flares[i] = 0   
        elif \
            flares[i] < 3.0*fluxC.std() and \
            flares[i-2] == 0 and \
            flares[i+1] == 0 :
                flares[i] = 0
        elif \
            flares[i] < 3.0*fluxC.std() and \
            flares[i-1] == 0 and \
            flares[i+2] == 0 :
                flares[i] = 0
    for i in range(len(flares)-1):        
        if  flares[i-1] == 0 and \
            flares[i+1] == 0 :
                flares[i] = 0


    R = 0.569
    Rsun = 6.96e08
    sigma = 5.67e-8
    Teff = 3702
    L = 4*np.pi*(R*Rsun)**2*sigma*(Teff**4)*(10**7)
    #counting flares and find their position
    flarecount = []
    flareposition = []

    def findlocalmax(queue):
        maximum = 0
        for i in range(len(queue)):
            if (queue[i] > maximum):
                maximum = queue[i]
                max_position = len(queue) - i
        return maximum, max_position

    is_zero = 1
    queue = []

    for i in range(len(flares)):
        if ((not is_zero) and flares[i] == 0):
            is_zero = 1

            #print (findlocalmax(queue))
            #find flareposition
            index = i - findlocalmax(queue)[1]
            flareposition.append(index)

            #sum the each point of the data in one flare
            total = sum(queue)
            flarecount.append(np.log10(total*30*60*L))

            queue = []
            continue
        if (is_zero and flares[i] != 0):
            is_zero = 0
        queue.append(flares[i])

    #print flareposition
    #print flarecount
    #print L
    #print len(flareposition)

    tup = (re.findall('\d+', s)[0],len(flareposition))
    countlist.append(tup)
    print s

#     plt.plot(time,flares,'.-')
#     plt.show()

#     plt.plot(time,100.*norm(flux_det[mnanm])-20,'b.-')
#     plt.plot(time,100.*norm(flux_det[mnanm])-100.*norm(trend_p[mnanm]),'k.-')

#     plt.plot(time,flux_d+20,'g.-')


#     plt.show()
    
print countlist




EPIC_211160943_mast.fits.rdb
EPIC_206305683_mast.fits.rdb
EPIC_217509281_mast.fits.rdb
EPIC_219306155_mast.fits.rdb
EPIC_219057545_mast.fits.rdb
EPIC_211997901_mast.fits.rdb
EPIC_211435030_mast.fits.rdb
EPIC_211711533_mast.fits.rdb
EPIC_211792422_mast.fits.rdb
EPIC_211801135_mast.fits.rdb
EPIC_211028347_mast.fits.rdb
EPIC_206494490_mast.fits.rdb
EPIC_211634376_mast.fits.rdb
EPIC_212022746_mast.fits.rdb
EPIC_206381460_mast.fits.rdb
EPIC_212531681_mast.fits.rdb
EPIC_212430908_mast.fits.rdb
EPIC_212814999_mast.fits.rdb
EPIC_212614976_mast.fits.rdb
EPIC_214733184_mast.fits.rdb
EPIC_211046195_mast.fits.rdb
EPIC_211963497_mast.fits.rdb
EPIC_210911931_mast.fits.rdb
EPIC_214394595_mast.fits.rdb
EPIC_212694426_mast.fits.rdb
EPIC_211494804_mast.fits.rdb
EPIC_211962038_mast.fits.rdb
EPIC_213826072_mast.fits.rdb
EPIC_210311375_mast.fits.rdb
EPIC_212716937_mast.fits.rdb
EPIC_219745741_mast.fits.rdb
EPIC_211386044_mast.fits.rdb
EPIC_211628806_mast.fits.rdb
EPIC_212411722_mast.fits.rdb
EPIC_210879793

In [22]:
KID,flare_count = map(list, zip(*countlist))
fdata = Table([KID, flare_count], names=['KID','flare_count'])
ascii.write(fdata,'/home/vatsal/UThesis/K2/data/kepler/ktwo-detrended/flare-count.rdb' , format='rdb')


In [34]:
print flare_count


hist, bins = np.histogram(flare_count, bins=50)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
plt.xlabel('Number of flares')
plt.ylabel('Number of M dwarfs')
plt.show()

[0, 3, 6, 14, 21, 2, 2, 21, 15, 0, 12, 28, 22, 1, 12, 5, 14, 13, 10, 4, 11, 2, 16, 9, 1, 7, 24, 51, 9, 8, 11, 3, 16, 8, 16, 7, 3, 3, 0, 11, 5, 8, 8, 12, 10, 23, 2, 5, 11, 10, 0, 15, 0, 16, 1, 6, 1, 6, 8, 1, 10, 0, 18, 14, 0, 0, 15, 5, 12, 1, 1, 6, 4, 6, 26, 2, 8, 8, 5, 14, 5, 12, 8, 2, 18, 0, 4, 20, 23, 12, 12, 8, 17, 18, 1, 11, 16, 1, 8, 16, 15, 10, 16, 12, 39, 17, 6, 2, 9, 15, 11, 4, 20, 13, 10, 14, 9, 17, 3, 4, 5, 25, 9, 13, 15, 1, 2, 11, 7, 3, 5, 6, 0, 3, 1, 1, 1, 3, 2, 16, 8, 22, 23, 15, 8, 4, 20, 14, 6, 4, 20, 7, 5, 7, 4, 0, 15, 14, 13, 6, 12, 8, 4, 7, 2, 10, 1, 11, 3, 8, 7, 5, 5, 9, 10, 11, 9, 2, 10, 15, 5, 16, 19, 2, 19, 20, 11, 9, 15, 5, 9, 26, 13, 26, 5, 18, 12, 12, 17, 25, 20, 9, 12, 20, 2, 12, 4, 8, 1, 14, 14, 16, 24, 0, 19, 3, 0, 2, 9, 12, 6, 14, 5, 4, 13, 7, 5, 15, 12, 0, 24, 0, 12, 7, 18, 10, 5, 11, 15, 1, 15, 2, 18, 6, 3, 18, 1, 16, 6, 1, 18, 1, 8, 13, 14]


In [36]:
print np.sum(flare_count)

2484
